In [ ]:
"""Train: here we build a train pipline, e.g. CIFAR 10 """

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras

In [2]:
""" 1&2 Dataset and Dataloader
"""
(x_train, y_train), (x_test, y_test)  = tf.keras.datasets.cifar10.load_data()
print(f"CIFAR10 x_train shape:{x_train.shape}, y_train shape : {y_train.shape}")
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(50000, 32, 32, 3).astype("float32") / 255, y_train)  # not reshape ot 1024*3
)
BATCH_SIZE = 64
dataloader = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

CIFAR10 x_train shape:(50000, 32, 32, 3), y_train shape : (50000, 1)


In [34]:
"""3 model
    1 build own layer
    2 build Conv layer
    """
class Hypercolum_layer(keras.layers.Layer):
    def __init__(self):
      super(Hypercolum_layer,self).__init__
    #   self.filters = 64
    #   self.kernel_size = (3,3)
    #   self.activation = tf.nn.relu
    
    def call(self, inputs):
        # LCN_1 =  keras.layers.Conv2D(64, kernel_size=(3,3), activation=tf.nn.relu)
        # x = LCN_1(inputs)
 
        return inputs

#! 定义理解错了，定义model和定义layer形式不同，在layer中直接增加loss即可
class Hypercolum_layer(keras.layers.Layer):
    """一个把activity稀疏正则化加入损失函数的Layer。"""

    def __init__(self, rate=1e-2):
        super(Hypercolum_layer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        # 使用`add_loss`来添加正则化损失函数，用输入的张量计算
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        LCN_1 =  keras.layers.Conv2D(64, kernel_size=(3,3), activation=tf.nn.relu)  #! 不是在call中建模的 加入__init__再调用或许可以
        x = LCN_1(inputs)
        return x


class CNN_3L(keras.layers.Layer):
    """3 layers CNN."""

    def __init__(self):
        super(CNN_3L, self).__init__() 
        self.conv_1 = keras.layers.Conv2D(64,  kernel_size=(3,3), activation=tf.nn.relu) # 
        self.flatten = keras.layers.Flatten(data_format='channels_last')
        self.linear = keras.layers.Dense(10)

    def call(self, inputs):
        x = self.conv_1(inputs)
        # x = tf.nn.relu(x)
        x = self.flatten(x)
        # x = tf.nn.relu(x)
        self.add_loss(tf.reduce_sum(inputs))  #TODO 修改loss即可完成Spatial Loss的定义
        return self.linear(x)

model= CNN_3L()

# 2 functional stack

"""

inputs = tf.keras.Input(shape=(32,32,3))  # ,batch_size=64
# d = tf.keras.layers.Dense(10)
# LCN_1 = keras.layers.Conv2D(64,  kernel_size=(3,3), activation=tf.nn.relu)
LCN_1 =  keras.layers.LocallyConnected2D(64, kernel_size=(3,3), activation=tf.nn.relu)
x = LCN_1(inputs)
Flatten_1 = keras.layers.Flatten(data_format='channels_last')
x = Flatten_1(x)
outputs = tf.keras.layers.Dense(10)(x)


model = tf.keras.Model(inputs, outputs)



# Weight regularization.
print(LCN_1.kernel.shape)  # LCN weight shape (900, 27, 64)(position_num,kernel3*3*3,channels), while CNN weight shape is  (3, 3, 3, 64)(image_channel, kernelsize1&2,weight channels)
#!current reshape way may not be right, better the weight be square
# 1 reshape to 2d sheet 
position_num,kernel_size_with_c,weight_channels = LCN_1.kernel.shape
sheet_2D_kernel_LCN_1 = tf.reshape(LCN_1.kernel, (30*3*8, 30*9*8)) #  ((position_num**0.5)*kernel_size_with_c*weight_channels, (position_num**0.5)*kernel_size_with_c*weight_channels)
# sheet_2D_kernel_LCN_1 = LCN_1.kernel.reshape((position_num**0.5)*kernel_size_with_c*weight_channels, (position_num**0.5)*kernel_size_with_c*weight_channels)  # (30*27*64, 30*27*64)
print(f"after reshape to 2D sheet:{sheet_2D_kernel_LCN_1.shape}")
top_left_LCN_1 = sheet_2D_kernel_LCN_1[:-1,:-1]
bottom_right_LCN_1 = sheet_2D_kernel_LCN_1[1:, 1:]
cosine_similarity_LCN_1 = np.dot(top_left_LCN_1, tf.transpose(bottom_right_LCN_1))/(np.linalg.norm(top_left_LCN_1)*np.linalg.norm(bottom_right_LCN_1))
loss_hypercolum = tf.reduce_mean((1-cosine_similarity_LCN_1)/2)
print(f"loss_hypercolum:{loss_hypercolum}")
# 2 sheet implement equation
alpha = 0.1
model.add_loss(lambda: alpha*loss_hypercolum)  # tf.reduce_mean(sheet_2D_kernel_LCN_1)
"""


# 3 Sequential
# model = keras.Sequential(
#     [
#         # keras.layers.Conv2D(64,  kernel_size=(3,3), activation=tf.nn.relu), # input_shape=(3, 32, 32),
#         # keras.layers.Conv2D(64, kernel_size=(3,3), activation=tf.nn.relu),
#         # keras.layers.Conv2D(64, kernel_size=(3,3), activation=tf.nn.relu),
#         keras.layers.LocallyConnected2D(64, kernel_size=(3,3), activation=tf.nn.relu),
#         # keras.layers.LocallyConnected2D(64, kernel_size=(3,3), activation=tf.nn.relu),
#         # keras.layers.LocallyConnected2D(64, kernel_size=(3,3), activation=tf.nn.relu),
#         keras.layers.Flatten(data_format='channels_last'),
#         keras.layers.Dense(10),
#     ]
# )

#fixdeTODO add spatial loss
"""no need to change LCN strcture, only need to add new loss
    steps 1 : lean how add loss
          2 : create own spatial loss: reshape weight to 2D sheet, the loss value = neighbour similarity equation
          3 : apply final loss = neighbour similarity + calssification
"""

'no need to change LCN strcture, only need to add new loss\n    steps 1 : lean how add loss\n          2 : create own spatial loss: reshape weight to 2D sheet, the loss value = neighbour similarity equation\n          3 : apply final loss = neighbour similarity + calssification\n'

In [35]:
"""2.0 train and test in keras way """
model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])


model.compile(optimizer="SGD", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),metrics=['accuracy'])


reshaped_x_train = x_train.reshape(50000, 32, 32, 3).astype("float32") / 255
print(reshaped_x_train.shape, y_train.shape)
model.fit(reshaped_x_train , y_train, epochs=1) # ,batch_size=64
# ! why is (64,10)? (batch_size,1) is label
model.summary()

# 3 CNN 24.25% 7.4 s
# NCN+LOSS 10.44%  39.9  # HOW TO GET 18.8 BEFORE?
# WITHOOUT LOSS SIMILAR TIME FOR ONE layer

(50000, 32, 32, 3) (50000, 1)
1563/1563 [==============================] - 5s 4ms/step - loss: 46527.8984 - accuracy: 0.2511
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_3l_16 (CNN_3L)           (None, 10)                577802    
_________________________________________________________________
softmax_13 (Softmax)         (None, 10)                0         
Total params: 577,802
Trainable params: 577,802
Non-trainable params: 0
_________________________________________________________________


In [8]:

"""4-5 Optimize and train the model in 2 style:
    1.0 tensorflow style
    2.0 Keras style
"""


"""1.0 tensorflow style """
# 4 optimzer
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Whether `y_pred` is expected to be a logits tensor

#TODO ADD hypercolumn to LCN and loss


# 5.0 Train 
from tqdm import tqdm
epoch_num = 1
for epoch in range(epoch_num):
    with tqdm(dataloader) as train_bar:
        for step, (x,y) in enumerate(train_bar):
            
            with tf.GradientTape() as tape:  # in torch, without this GradientTape
                
                #! forward, edfault gradien zero?
                loss = loss_fn(y, model(x)) # more with zero_grad
                loss += sum(model.losses)# loss.backward() in torch
                gradient = tape.gradient(loss, model.trainable_weights) # in torch training paras setting in optim
            
            optimizer.apply_gradients(zip(gradient, model.trainable_weights))# in torch optimizer.step()
    
            if step % 100 == 0:
                print("Epoch",epoch, "Step:", step, "Loss:", float(loss))


# time = str(datetime.now()).split(" ")[0].replace("-", "_")
# torch.save(model.state_dict(), "model_{}.pth".format(time))

  0%|          | 2/782 [00:01<05:49,  2.23it/s]

Epoch 0 Step: 0 Loss: 2.3522157669067383


 13%|█▎        | 102/782 [00:20<01:59,  5.70it/s]

Epoch 0 Step: 100 Loss: 2.352651834487915


 26%|██▌       | 202/782 [00:41<01:55,  5.03it/s]

...

 77%|███████▋  | 602/782 [01:57<00:32,  5.59it/s]

Epoch 0 Step: 600 Loss: 2.3525807857513428


 90%|████████▉ | 702/782 [02:16<00:13,  5.77it/s]

Epoch 0 Step: 700 Loss: 2.352726697921753


100%|██████████| 782/782 [02:32<00:00,  5.14it/s]


In [6]:
# predict
model.summary()  # pritn(structure)

print("Epoch",epoch, "Step:", step, "Loss:", float(loss))
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
Predictions = probability_model.predict((x_train.reshape(50000, 32, 32, 3).astype("float32") / 255)[:100])
print(Predictions[5], np.argmax(Predictions[5]))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_3 (Functional)    (64, 10)                  2188810   
_________________________________________________________________
softmax_1 (Softmax)          (64, 10)                  0         
Total params: 2,188,810
Trainable params: 2,188,810
Non-trainable params: 0
_________________________________________________________________
Epoch 0 Step: 781 Loss: 2.552854061126709
[0.10021344 0.09999754 0.09981877 0.09991436 0.10013708 0.09982315
 0.10009785 0.10008855 0.10007832 0.09983091] 0


In [17]:
"""2.0 train and test in keras way """
model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
model.summary()

model.compile(optimizer="SGD", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),metrics=['accuracy'])
reshaped_train = x_train.reshape(50000, 32, 32, 3).astype("float32") / 255
print(reshaped_train.shape, y_train.shape)
model.fit(reshaped_train , y_train, epochs=1) # ,batch_size=64
# ! why is (64,10)? (batch_size,1) is label


# 3 CNN 24.25% 7.4 s
# NCN+LOSS 10.44%  39.9  # HOW TO GET 18.8 BEFORE?
# WITHOOUT LOSS SIMILAR TIME FOR ONE layer

AttributeError: 'Hypercolum_layer' object has no attribute '_trainable'

In [6]:
# predict
# model.summary()  # pritn(structure)

# _,(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# print("Epoch",epoch, "Step:", step, "Loss:", float(loss))
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
Predictions = probability_model.predict(x_train[:100])
print(Predictions[5], np.argmax(Predictions[5]))
                                         

313/313 - 1s - loss: 2.2167 - accuracy: 0.2442

Test accuracy: 0.24420000612735748
[0.08533674 0.08533674 0.08533674 0.08533674 0.08533674 0.08533674
 0.08533674 0.08533674 0.08533674 0.23196931] 9
